In [1]:
import pathlib
import os
import pickle
import collections
import timeit

import optimum
import optimum.onnxruntime
import onnxruntime
import datasets
import torch
import torch.nn as nn
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import segmentador
import segmentador.optimize
import eval_model

%load_ext autoreload
%autoreload 2

DEV_RUN = True

QUANTIZED_MODELS_DIR = "quantized_models"
pathlib.Path(QUANTIZED_MODELS_DIR).mkdir(exist_ok=True, parents=True)

In [2]:
curated_df = datasets.Dataset.from_file(
    "../data/refined_datasets/df_tokenized_split_0_120000_6000/"
    "combined_test_48_parts_1036_instances/dataset.arrow"
)

curated_df_subsample = curated_df.shard(num_shards=200, index=0)

## Creating LSTM Quantization

In [3]:
segmenter_lstm = segmentador.LSTMSegmenter(
    uri_model="../pretrained_segmenter_model/512_6000_1_lstm/checkpoints/epoch=3-step=3591.ckpt",
    uri_tokenizer="../tokenizers/6000_subwords",
    device="cpu",
)

### With Torch format

In [4]:
quantized_lstm_torch_paths = segmentador.optimize.quantize_model(
    segmenter_lstm,
    model_output_format="torch",
    check_cached=False,
    verbose=True,
)

Saved quantized Pytorch module in './quantized_models/q_num_layers_1_vocab_size_6000_lstm_model.pt'. To use it, load a LSTM segmenter model as:

LSTMSegmenter(
   uri_model='./quantized_models/q_num_layers_1_vocab_size_6000_lstm_model.pt',
   uri_tokenizer='../tokenizers/6000_subwords',
   quantize_weights=True,
   ...,
)


In [5]:
segmenter_lstm_torch_quantized = segmentador.LSTMSegmenter(
   uri_model=quantized_lstm_torch_paths.output_uri,
   uri_tokenizer='../tokenizers/6000_subwords',
   quantize_weights=True,
)

In [6]:
segmenter_lstm_torch_quantized(curated_df_subsample, show_progress_bar=True)

  0%|          | 0/1 [00:00<?, ?it/s]

['INDICAÇÃO Nº DE 2008 ( Do Sr. Hermes Parcianello )',
 'Indica ao Ministério da Educação a inclusão no ProInfância do município de LARANJEIRAS DO SUL, no estado do Paraná.',
 'Excelentíssimo Senhor Ministro de Estado da Educação, Considerando que a construção de creches e escolas de educação infantil e a reestruturação e aquisição de equipamentos para a rede física escolar desse nível educacional são indispensáveis à melhoria da educação. Considerando um grande percentual de mulheres chefes de família com filhos menores que necessitam de local adequado para deixar seus filhos e que o Programa disponibiliza o atendimento de até 120 crianças em período integral ou até 240 crianças, se distribuídos em turnos matutinos e vespertinos. Considerando que o município ao ser atendido pelo Programa priorizará a acessibilidade, fazendo as adequações necessárias a fim de permitir seu uso por portadores de necessidades especiais. Considerando que a edificação será composta de um bloco administrativ

### With ONNX format

In [16]:
quantized_lstm_onnx_paths = segmentador.optimize.quantize_model(
    segmenter_lstm,
    model_output_format="onnx",
    check_cached=False,
    verbose=True,
)

2022-04-09 01:11:05.924922671 [W:onnxruntime:, inference_session.cc:1546 Initialize] Serializing optimized model with Graph Optimization level greater than ORT_ENABLE_EXTENDED and the NchwcTransformer enabled. The generated model may contain hardware specific optimizations, and should only be used in the same environment the model was optimized in.


Saved quantized Pytorch module (ONNX format) in './quantized_models/q_hidden_layer_dim_512_vocab_size_6000_num_layers_1_opt_level_99_lstm_model.onnx'. To use it, load a LSTM segmenter model as:

segmentador.optimize.ONNXLSTMSegmenter(
   uri_model='./quantized_models/q_hidden_layer_dim_512_vocab_size_6000_num_layers_1_opt_level_99_lstm_model.onnx',
   uri_tokenizer='../tokenizers/6000_subwords',
   ...,
)


In [17]:
segmenter_lstm_onnx_quantized = segmentador.optimize.ONNXLSTMSegmenter(
   uri_model=quantized_lstm_onnx_paths.output_uri,
   uri_tokenizer='../tokenizers/6000_subwords',
)

In [18]:
segmenter_lstm_onnx_quantized(curated_df_subsample, show_progress_bar=True)

  0%|          | 0/1 [00:00<?, ?it/s]

['INDICAÇÃO Nº DE 2008 ( Do Sr. Hermes Parcianello )',
 'Indica ao Ministério da Educação a inclusão no ProInfância do município de LARANJEIRAS DO SUL, no estado do Paraná.',
 'Excelentíssimo Senhor Ministro de Estado da Educação, Considerando que a construção de creches e escolas de educação infantil e a reestruturação e aquisição de equipamentos para a rede física escolar desse nível educacional são indispensáveis à melhoria da educação. Considerando um grande percentual de mulheres chefes de família com filhos menores que necessitam de local adequado para deixar seus filhos e que o Programa disponibiliza o atendimento de até 120 crianças em período integral ou até 240 crianças, se distribuídos em turnos matutinos e vespertinos. Considerando que o município ao ser atendido pelo Programa priorizará a acessibilidade, fazendo as adequações necessárias a fim de permitir seu uso por portadores de necessidades especiais. Considerando que a edificação será composta de um bloco administrativ

## Creating BERT Quantization

In [10]:
segmenter_bert = segmentador.BERTSegmenter(
    uri_model="../pretrained_segmenter_model/4_6000_layer_model/",
    device="cpu",
)

In [11]:
quantized_bert_paths = segmentador.optimize.quantize_model(
    segmenter_bert,
    check_cached=False,
    verbose=True,
)

2022-04-09 01:08:16.644658683 [W:onnxruntime:, inference_session.cc:1546 Initialize] Serializing optimized model with Graph Optimization level greater than ORT_ENABLE_EXTENDED and the NchwcTransformer enabled. The generated model may contain hardware specific optimizations, and should only be used in the same environment the model was optimized in.


failed in shape inference <class 'AssertionError'>
failed in shape inference <class 'AssertionError'>
Saved quantized BERT (ONNX format) in './quantized_models/q_num_layers_4_vocab_size_6000_opt_level_99_bert_model.onnx', and its configuration file in './quantized_models/q_num_layers_4_vocab_size_6000_opt_level_99_bert_model.onnx.config'. To use it, load a BERT segmenter model as:

segmentador.optimize.ONNXBERTSegmenter(
   uri_model='./quantized_models/q_num_layers_4_vocab_size_6000_opt_level_99_bert_model.onnx',
   uri_tokenizer='../pretrained_segmenter_model/4_6000_layer_model/',
   uri_onnx_config='./quantized_models/q_num_layers_4_vocab_size_6000_opt_level_99_bert_model.onnx.config',
   ...,
)


In [12]:
segmenter_bert_quantized = segmentador.optimize.ONNXBERTSegmenter(
   uri_model=quantized_bert_paths.output_uri,
   uri_tokenizer='../pretrained_segmenter_model/4_6000_layer_model/',
   uri_onnx_config=quantized_bert_paths.onnx_config_uri,
)

In [13]:
segmenter_bert_quantized(curated_df_subsample, show_progress_bar=True)

  0%|          | 0/1 [00:00<?, ?it/s]

['INDICAÇÃO Nº DE 2008 ( Do Sr. Hermes Parcianello )',
 'Indica ao Ministério da Educação a inclusão no ProInfância do município de LARANJEIRAS DO SUL, no estado do Paraná.',
 'Excelentíssimo Senhor Ministro de Estado da Educação, Considerando que a construção de creches e escolas de educação infantil e a reestruturação e aquisição de equipamentos para a rede física escolar desse nível educacional são indispensáveis à melhoria da educação. Considerando um grande percentual de mulheres chefes de família com filhos menores que necessitam de local adequado para deixar seus filhos e que o Programa disponibiliza o atendimento de até 120 crianças em período integral ou até 240 crianças, se distribuídos em turnos matutinos e vespertinos. Considerando que o município ao ser atendido pelo Programa priorizará a acessibilidade, fazendo as adequações necessárias a fim de permitir seu uso por portadores de necessidades especiais. Considerando que a edificação será composta de um bloco administrativ

## Validating performance of distinct configurations

In [14]:
def validate(
    model,
    moving_window_size: int = 1024,
    window_shift_size: float = 0.5,
    batch_size: int = 64,
) -> dict[str, float]:
    sharded_df = curated_df.shard(num_shards=5, index=0)
    
    t_start = timeit.time.perf_counter()
    
    logits = model(
        sharded_df,
        batch_size=batch_size,
        return_logits=True,
        show_progress_bar=True,
        window_shift_size=window_shift_size,
        moving_window_size=moving_window_size,
    ).logits
    
    t_delta = timeit.time.perf_counter() - t_start
    
    metrics = eval_model.compute_metrics(
        ([logits], [np.concatenate(sharded_df["labels"])]),
    )
    metrics["approx_inference_time"] = t_delta
    
    return metrics

In [15]:
quantized_bert_paths = segmentador.optimize.quantize_model(segmenter_bert)